<a href="https://colab.research.google.com/github/KseniyaShilova/rl-game/blob/main/rlGameVideo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from collections import deque
import numpy as np
import torch
#from gym import make
from torch import nn
from torch.optim import Adam, lr_scheduler

In [2]:
GAMMA = 0.99
INITIAL_STEPS = 1024
TRANSITIONS = 4_444
STEPS_PER_UPDATE = 5
STEPS_PER_TARGET_UPDATE = STEPS_PER_UPDATE * 1000
BATCH_SIZE = 256
BUFFER_SIZE = 85_000
LEARNING_RATE =  0.000001
DEVICE = torch.device("cuda")

In [3]:
!git clone https://github.com/Farama-Foundation/Arcade-Learning-Environment/

Cloning into 'Arcade-Learning-Environment'...
remote: Enumerating objects: 9404, done.
remote: Counting objects: 100% (1199/1199), done.
remote: Compressing objects: 100% (507/507), done.
remote: Total 9404 (delta 770), reused 1002 (delta 662), pack-reused 8205
Receiving objects: 100% (9404/9404), 7.68 MiB | 23.20 MiB/s, done.
Resolving deltas: 100% (7242/7242), done.


In [4]:
#!git clone https://github.com/Farama-Foundation/gym-examples
# cd gym-examples

# source .env/bin/activate
#!pip install -e .

In [5]:
# from gym.envs.registration import register

# register(
#     id="gym_examples/GridWorld-v0",
#     entry_point="gym_examples.envs:GridWorldEnv",
# )

In [6]:
!pip install gymnasium[atari]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.3 MB/s eta 0:00:00


In [7]:
!pip install gymnasium[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=5291e48246e4e4f110116e8ff57b936971db56a0f14d56f55f00bdb108c32bbd
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [8]:
!pip install Gym[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 14.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  not

In [9]:
!pip install ufal.pybox2d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 38.9 MB/s eta 0:00:00


In [10]:
from ale_py import ALEInterface, SDL_SUPPORT

In [11]:
!pip install "gym[atari, accept-rom-license]"

  Using cached ale_py-0.7.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.8.1
    Uninstalling ale-py-0.8.1:
      Successfully uninstalled ale-py-0.8.1


In [12]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]

In [13]:
!pip install gym[atari,accept-rom-license]==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [14]:
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

  Using cached ale_py-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.7.5
    Uninstalling ale-py-0.7.5:
      Successfully uninstalled ale-py-0.7.5


In [15]:
!pip install --upgrade git+https://github.com/openai/gym
!pip install autorom
#AutoRom
!pip install --upgrade gym[atari]

  Cloning https://github.com/openai/gym to /tmp/pip-req-build-ckws1dxj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/gym /tmp/pip-req-build-ckws1dxj
  Resolved https://github.com/openai/gym to commit dcd185843a62953e27c2d54dc8c2d647d604b635
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827619 sha256=86ea05a6f60d730df34f80e4a654db30a7e6ff3fc70df6ce5f0bab8c0ff77613
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ipmrm2x/wheels/d1/2b/89/4a44b1366d1bc84847d0afaeb7a5cb5b8043d2d73660a49512
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followin

In [16]:
import ale_py
import shimmy

import gym
import gymnasium

In [17]:
!pip install -U gym

In [18]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]

In [19]:
# import gym
import numpy as np


class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)

In [20]:
from gym.spaces import Discrete


class DiscreteActions(gym.ActionWrapper):
    def __init__(self, env, disc_to_cont):
        super().__init__(env)
        self.disc_to_cont = disc_to_cont
        self.action_space = Discrete(len(disc_to_cont))

    def action(self, act):
        return self.disc_to_cont[act]

In [21]:
class ReacherRewardWrapper(gym.Wrapper):
    def __init__(self, env, reward_dist_weight, reward_ctrl_weight):
        super().__init__(env)
        self.reward_dist_weight = reward_dist_weight
        self.reward_ctrl_weight = reward_ctrl_weight

    def step(self, action):
        obs, _, terminated, truncated, info = self.env.step(action)
        reward = (
            self.reward_dist_weight * info["reward_dist"]
            + self.reward_ctrl_weight * info["reward_ctrl"]
        )
        return obs, reward, terminated, truncated, info

In [22]:
import gym
from gym.spaces import Box
import numpy as np


class RelativePosition(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(shape=(2,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        return obs["target"] - obs["agent"]

In [23]:
import numpy as np
import pygame



class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None, size=5):
        self.size = size  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(4)

        """
        The following dictionary maps abstract actions from `self.action_space` to
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

In [24]:
def _get_obs(self):
    return {"agent": self._agent_location, "target": self._target_location}

In [25]:
def _get_info(self):
    return {
        "distance": np.linalg.norm(
            self._agent_location - self._target_location, ord=1
        )
    }

In [26]:
def reset(self, seed=None, options=None):
    # We need the following line to seed self.np_random
    super().reset(seed=seed)

    # Choose the agent's location uniformly at random
    self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)

    # We will sample the target's location randomly until it does not coincide with the agent's location
    self._target_location = self._agent_location
    while np.array_equal(self._target_location, self._agent_location):
        self._target_location = self.np_random.integers(
            0, self.size, size=2, dtype=int
        )

    observation = self._get_obs()
    info = self._get_info()

    if self.render_mode == "human":
        self._render_frame()

    return observation, info

In [27]:
def step(self, action):
    # Map the action (element of {0,1,2,3}) to the direction we walk in
    direction = self._action_to_direction[action]
    # We use `np.clip` to make sure we don't leave the grid
    self._agent_location = np.clip(
        self._agent_location + direction, 0, self.size - 1
    )
    # An episode is done iff the agent has reached the target
    terminated = np.array_equal(self._agent_location, self._target_location)
    reward = 1 if terminated else 0  # Binary sparse rewards
    observation = self._get_obs()
    info = self._get_info()

    if self.render_mode == "human":
        self._render_frame()

    return observation, reward, terminated, False, info

In [28]:
def render(self):
    if self.render_mode == "rgb_array":
        return self._render_frame()

def _render_frame(self):
    if self.window is None and self.render_mode == "human":
        pygame.init()
        pygame.display.init()
        self.window = pygame.display.set_mode(
            (self.window_size, self.window_size)
        )
    if self.clock is None and self.render_mode == "human":
        self.clock = pygame.time.Clock()

    canvas = pygame.Surface((self.window_size, self.window_size))
    canvas.fill((255, 255, 255))
    pix_square_size = (
        self.window_size / self.size
    )  # The size of a single grid square in pixels

    # First we draw the target
    pygame.draw.rect(
        canvas,
        (255, 0, 0),
        pygame.Rect(
            pix_square_size * self._target_location,
            (pix_square_size, pix_square_size),
        ),
    )
    # Now we draw the agent
    pygame.draw.circle(
        canvas,
        (0, 0, 255),
        (self._agent_location + 0.5) * pix_square_size,
        pix_square_size / 3,
    )

    # Finally, add some gridlines
    for x in range(self.size + 1):
        pygame.draw.line(
            canvas,
            0,
            (0, pix_square_size * x),
            (self.window_size, pix_square_size * x),
            width=3,
        )
        pygame.draw.line(
            canvas,
            0,
            (pix_square_size * x, 0),
            (pix_square_size * x, self.window_size),
            width=3,
        )

    if self.render_mode == "human":
        # The following line copies our drawings from `canvas` to the visible window
        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.update()

        # We need to ensure that human-rendering occurs at the predefined framerate.
        # The following line will automatically add a delay to keep the framerate stable.
        self.clock.tick(self.metadata["render_fps"])
    else:  # rgb_array
        return np.transpose(
            np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
        )

In [29]:
def close(self):
    if self.window is not None:
        pygame.display.quit()
        pygame.quit()

In [30]:
full_action_space=True
#

In [31]:
import numpy as np
from gym import ActionWrapper, ObservationWrapper, RewardWrapper, Wrapper

from gym.spaces import Box, Discrete


class RelativePosition(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(shape=(2,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        return obs["target"] - obs["agent"]

In [32]:
from typing import SupportsFloat


class ClipReward(RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, r: SupportsFloat) -> SupportsFloat:
        return np.clip(r, self.min_reward, self.max_reward)

In [33]:
class ReacherRewardWrapper(Wrapper):
    def __init__(self, env, reward_dist_weight, reward_ctrl_weight):
        super().__init__(env)
        self.reward_dist_weight = reward_dist_weight
        self.reward_ctrl_weight = reward_ctrl_weight

    def step(self, action):
        obs, _, terminated, truncated, info = self.env.step(action)
        reward = (
            self.reward_dist_weight * info["reward_dist"]
            + self.reward_ctrl_weight * info["reward_ctrl"]
        )
        return obs, reward, terminated, truncated, info

In [34]:
import gym
class DiscreteActions(ActionWrapper):
    def __init__(self, env, disc_to_cont):
        super().__init__(env)
        self.disc_to_cont = disc_to_cont
        self.action_space = Discrete(len(disc_to_cont))

    def action(self, act):
        return self.disc_to_cont[act]


if __name__ == "__main__":
    env = gymnasium.make("ALE/Freeway-v5")
    wrapped_env = DiscreteActions(
        env, [np.array([1, 0]), np.array([-1, 0]), np.array([0, 1]), np.array([0, -1])]
    )
    print(wrapped_env.action_space)  # Discrete(4)

Discrete(4)


cod for change

In [35]:
#cod

In [36]:
class ExpirienceReplay(deque):
    def sample(self, size):
        batch = random.sample(self, size)
        return list(zip(*batch))

In [37]:
from torchvision import transforms, models

In [38]:
class DQN:
    def __init__(self, state_dim, action_dim):
        self.steps = 0  # Do not change
        self.model = models.resnet18(pretrained=False, num_classes = action_dim).to(DEVICE)  # Torch model
        self.target_model = models.resnet18(pretrained=False, num_classes = action_dim).to(DEVICE)
        self.target_model.load_state_dict(self.model.state_dict())
        self.buffer = ExpirienceReplay(maxlen=BUFFER_SIZE)
        self.optimizer = Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.criteria = nn.MSELoss()
        self.scheduler = lr_scheduler.StepLR(
            self.optimizer, step_size=10000, gamma=0.8
        )
        self.transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

    def consume_transition(self, transition):
        state, action, next_state, reward, done = transition
        state = self.transforms(state).unsqueeze(0)
        next_state = self.transforms(next_state).unsqueeze(0)
        transition = state, action, next_state, reward, done
        self.buffer.append(transition)

    def sample_batch(self):
        batch = self.buffer.sample(BATCH_SIZE)
        state, action, next_state, reward, done = batch
        state = torch.cat(state, dim=0)
        action = torch.tensor(np.array(action, dtype=np.int64))
        next_state = torch.cat(next_state, dim=0)
        reward = torch.tensor(np.array(reward, dtype=np.float32))
        done = torch.tensor(np.array(done, dtype=np.int32))
        return state, action, next_state, reward, done

    def train_step(self, batch):
        if not self.model.training:
            self.model.train()
        self.optimizer.zero_grad()
        state, action, next_state, reward, done = batch
        current_q = self.model(state.to(DEVICE))
        next_q = self.model(state.to(DEVICE))
        next_action = torch.argmax(next_q, 1)
        next_target_q = self.target_model(next_state.to(DEVICE))
        action_reward = current_q.gather(1, action.view(-1, 1).to(DEVICE))
        next_actions_reward = next_target_q.gather(1, next_action.view(-1, 1))
        next_actions_reward = next_actions_reward.squeeze(1) * (1 - done.to(DEVICE))
        loss = self.criteria(
            action_reward.squeeze(1), reward.to(DEVICE) + GAMMA * next_actions_reward
        )
        loss.backward()
        self.optimizer.step()
        if self.steps > 1_000_000:
            self.scheduler.step(loss)

    def update_target_network(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def act(self, state, target=False):
        full_action_space=True
        if self.model.training:
            self.model.eval()
        network = self.target_model if target else self.model
        state = self.transforms(state).to(DEVICE)
        action_rewards = network(state.unsqueeze(0)).squeeze(0).detach().cpu().numpy()
        return np.argmax(action_rewards)

    def update(self, transition):
        self.consume_transition(transition)
        if self.steps % STEPS_PER_UPDATE == 0:
            batch = self.sample_batch()
            self.train_step(batch)
        if self.steps % STEPS_PER_TARGET_UPDATE == 0:
            self.update_target_network()
        self.steps += 1

    def save(self):
        torch.save(self.model.state_dict(), "agent.pkl")

In [39]:
def evaluate_policy(agent, episodes=5):
    env = gymnasium.make("ALE/Freeway-v5")
    returns = []
    agent.model.eval()
    for _ in range(episodes):
        done = False
        state = env.reset()[0]
        total_reward = 0.0

        while not done:
            state, reward, done, _, _ = env.step(agent.act(state))
            total_reward += reward
        returns.append(total_reward)
    agent.model.train()
    return returns

In [40]:
# dqn = DQN(state_dim=env.observation_space.shape[0], action_dim=env.action_space.n)
# torch.load(dqn, 'dqn.pt')

In [41]:
if __name__ == "__main__":
    env = gymnasium.make("ALE/Freeway-v5")
    dqn = DQN(state_dim=env.observation_space.shape[0], action_dim=env.action_space.n)
    eps = 0.1
    eps_decay = 2
    eps_min = 0.01

    state, _ = env.reset()

    for _ in range(INITIAL_STEPS):
        action = env.action_space.sample()

        next_state, reward, done, _, _ = env.step(action)
        dqn.consume_transition((state, action, next_state, reward, done))

        state = next_state if not done else env.reset()

    for i in range(TRANSITIONS):

        if i % 25_000 == 0:
            eps = max(eps / eps_decay, eps_min)

        if random.random() < eps:
            action = env.action_space.sample()
        else:
            action = dqn.act(state)

        next_state, reward, done, _, _ = env.step(action)
        dqn.update((state, action, next_state, reward, done))

        state = next_state if not done else env.reset()[0]

        if (i + 1) % (TRANSITIONS // 100) == 0:
            rewards = evaluate_policy(dqn, 5)
            print(
                f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}"
            )
            dqn.save()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Step: 44, Reward mean: 21.4, Reward std: 0.7999999999999999
Step: 88, Reward mean: 22.8, Reward std: 0.9797958971132713
Step: 132, Reward mean: 18.4, Reward std: 0.7999999999999999
Step: 176, Reward mean: 14.8, Reward std: 0.39999999999999997
Step: 220, Reward mean: 10.6, Reward std: 2.0591260281974
Step: 264, Reward mean: 6.2, Reward std: 1.32664991614216
Step: 308, Reward mean: 1.8, Reward std: 0.9797958971132712
Step: 352, Reward mean: 0.6, Reward std: 0.48989794855663565
Step: 396, Reward mean: 2.0, Reward std: 1.0954451150103321
Step: 440, Reward mean: 2.4, Reward std: 1.624807680927192
Step: 484, Reward mean: 4.2, Reward std: 2.1354156504062622
Step: 528, Reward mean: 7.4, Reward std: 1.9595917942265424
Step: 572, Reward mean: 9.0, Reward std: 1.6733200530681511
Step: 616, Reward mean: 10.0, Reward std: 1.4142135623730951
Step: 660, Reward mean: 10.0, Reward std: 2.1908902300206643
Step: 704, Reward mean: 10.8, Reward std: 1.7204650534085253
Step: 748, Reward mean: 11.0, Reward s

In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [43]:
torch.save(dqn, 'dqn.pt')

In [44]:
import cv2

fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('output1.avi', fourcc, 4.0, (next_state.shape[1], next_state.shape[0]))
state, _ = env.reset()
done = False
while not done:
    action = dqn.act(state)
    next_state, reward, done, _, _ = env.step(action)
    state = next_state
    out.write(next_state)
out.release()